In [1]:
!python3 -m pip install -e .

import random
import time
import copy
from collections import deque

import numpy as np

import torch as tc
import torch.optim as opt

import gym
import gym_snake
env = gym.make('snake-v0')

from model import NN
from core import CUDA_AVAILABLE, DEVICE

from per_memory import PERMemory

Obtaining file:///home/lobo/gym-snake
  Attempting uninstall: gym-snake
    Found existing installation: gym-snake 0.0.1
    Uninstalling gym-snake-0.0.1:
      Successfully uninstalled gym-snake-0.0.1
  Running setup.py develop for gym-snake


In [2]:
#Train Parameters

BATCH_SIZE = 32
DISC_RATIO=0.9
EPISODE_CNT=50000
EPS_EXPONENT=2.6
EPS_MAX=1.0
EPS_MIN=0.1
LEARN_FREQ=8
REPLAY_MEM_SIZE = 500000
STAT_DISPLAY_FREQ = 100
SAVE_TEMP_FREQ = 400
SAVE_VERSIONS_FREQ = EPISODE_CNT//10
TARGET_UPD_FREQ = 1000
VALIDATION_EPISODE_CNT = 100
VALIDATION_FREQ = EPISODE_CNT//100

In [3]:
from IPython.core.debugger import set_trace
#set_trace()

In [4]:
print("CUDA: ",CUDA_AVAILABLE)
net=NN().cuda() if CUDA_AVAILABLE else NN()
net_target=copy.deepcopy(net)
net_target.load_state_dict(net.state_dict())
net.train()
net_target.eval()

opter=opt.Adam(net.parameters(),lr=1e-4)
perm = PERMemory(REPLAY_MEM_SIZE)

def train():
    idxs,isws,bat=zip(*perm.sample(BATCH_SIZE,epi/EPISODE_CNT))
    bat=zip(*bat)
    s1bat,abat,rbat,s2bat,dbat = map(lambda x:tc.tensor(x).to(DEVICE),bat)

    q1=net(s1bat)
    with tc.no_grad():
        q2=net_target(s2bat)
    x=q1.gather(1,abat.unsqueeze(dim=1)).squeeze()
    y=rbat+DISC_RATIO*((1-dbat)*tc.max(q2,dim=1)[0])
    
    #MSE Loss with Importance Sampling Weight
    loss=(tc.tensor(isws).to(DEVICE)*(x-y)**2).mean()
    opter.zero_grad()
    loss.backward()
    opter.step()
    losses.append(loss.cpu().detach().numpy())

eps=EPS_MAX
losses=[]
rwdsums=[]
for epi in range(1,EPISODE_CNT+1):
    s1=env.reset()
    done=False
        
    rwdsum=0
    step=1
    while not done:
        q1=net(tc.tensor(s1).to(DEVICE))
        actidx=(np.random.randint(0,3) if random.random()<eps else 
                np.argmax(q1.cpu().detach().numpy()))
        val1=q1[0][actidx]
        s2,rwd,done,info=env.step(actidx)
        with tc.no_grad():
            q2=net_target(tc.tensor(s2).to(DEVICE))
        val2=rwd+DISC_RATIO*((1-done)*tc.max(q2,dim=1)[0])
        td=val2-val1
        perm.push(td,(s1,actidx,rwd,s2,int(done)))
        
        s1=s2
        rwdsum+=rwd
        
        if step%LEARN_FREQ==0 and perm.cnt>=BATCH_SIZE:
            train()
        if step%TARGET_UPD_FREQ==0:
            net_target.load_state_dict(net.state_dict())
            net_target.eval()
        step+=1
            
    rwdsums.append(rwdsum)
    eps=min(EPS_MAX,max(EPS_MIN, ((EPISODE_CNT-epi)/EPISODE_CNT)**EPS_EXPONENT ))
        
    if epi%SAVE_TEMP_FREQ==0:
        tc.save(net.state_dict(),'./netw.pt')
        from IPython.display import clear_output
        clear_output(wait=True)
    if epi%STAT_DISPLAY_FREQ==0:
        print("{}/{}({:.2f}%): LossAvg={:.4f} RwdAvg={:.4f}".format(
            epi,
            EPISODE_CNT,
            epi/EPISODE_CNT*100,
            sum(losses)/len(losses),
            sum(rwdsums)/len(rwdsums)))
        rwdsums=[]
        losses=[]


#save
tc.save(net.state_dict(),'./netw.pt')
print("DONE!!!")

4800/50000(9.60%): LossAvg=0.0005 RwdAvg=-0.0610
4900/50000(9.80%): LossAvg=0.0006 RwdAvg=-0.0593
5000/50000(10.00%): LossAvg=0.0006 RwdAvg=-0.0580
5100/50000(10.20%): LossAvg=0.0006 RwdAvg=-0.0597


KeyboardInterrupt: 

In [ ]:
TODO: MCTS적용할거니까 4방향 액션 쓰자. 그게 학습이 더 잘되는듯.